In [1]:
import numpy as np
from PIL import  Image, ImageDraw, ImagePath 
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image as IPython_display_Image
from IPython.display import display as IPython_display
import math

In [2]:
df_pieces = pd.read_csv(f"..\\data\\group_45\\final_transforms.csv")
df_pieces.head()

,piece,t_x,t_y,r_sin,r_cos
0,0,0.82156,-0.269809,-0.907949,0.419080
1,1,-1.52998,0.894071,0.873700,0.486464


In [3]:
tranformations = df_pieces.values.tolist()
tranformations[:5]

[[0.0, 0.82156, -0.269809, -0.907949, 0.41908],
 [1.0, -1.52998, 0.894071, 0.8737, 0.486464]]

In [4]:
def transparent_image(piece_image, translation_vector, sin_theta, cos_theta, background_image):
    # Get the size of the original image
    original_width, original_height = piece_image.size

    # Get the size of the background image
    bg_width, bg_height = background_image.size

    # Rotate the original image
    #rotated_piece_image = piece_image.rotate(-np.degrees(np.arccos(cos_theta)), resample=Image.BILINEAR, expand=True)
    rotated_piece_image = piece_image.rotate(-np.degrees(np.arccos(cos_theta)))#, resample=Image.BILINEAR, expand=True)

    # Calculate the position to paste the rotated image in the new image
    paste_position = ((bg_width - original_width) // 2 + int(translation_vector[0]),
                      (bg_height - original_height) // 2 + int(translation_vector[1]))

    # Paste the rotated image onto the background image
    #background_image.paste(rotated_piece_image, paste_position, rotated_piece_image)
    background_image.paste(rotated_piece_image, paste_position)
    # Return the resulting background image with the updated piece image
    return background_image

In [13]:
background = Image.new('RGBA', (4885, 4300), (0,0,0,0))
angle = 0 #np.pi/2 # 0
num_fragments = 2
images_paths = [f"..\\data\\group_45\\RPf_00368_intact_mesh.png",f"..\\data\\group_45\\RPf_00370_intact_mesh.png"]

for image_path,trans in zip(images_paths[:num_fragments],tranformations[:num_fragments]):
    piece_id = int(trans[0])
    t_x = trans[1]*500
    t_y = trans[2]*500
    translation_vector = [t_x,t_y] #(int(trans[1])*1000,int(trans[2])*1000)

    sin_theta = trans[3] ##np.sin(angle) #trans[3]
    cos_theta = trans[4] ##np.cos(angle) #trans[4]

    #piece_image = Image.open(f"..\\data\\ofir\\Pseudo-Sappho_MAN_Napoli_Inv9084\\Puzzle1\\0\\{piece_id}_.png")
    piece_image = Image.open(image_path)
    background = transparent_image(piece_image,translation_vector,sin_theta,cos_theta,background)


background.show()

In [16]:
#from PIL import Image
#

def paste_images(image1, image2, x_offset, y_offset, rotation1=0, rotation2=0):
    # Open the images
    img1 = Image.open(image1)
    img2 = Image.open(image2)

    # Rotate the images
    img1 = img1.rotate(rotation1, expand=True)
    img2 = img2.rotate(rotation2, expand=True)

    # Create a transparent background image
    bg = Image.new('RGBA', (max(img1.width, img2.width), max(img1.height, img2.height)), (0, 0, 0, 0))

    # Paste the first image onto the transparent background
    bg.paste(img1, (0, 0))

    # Calculate the x and y offsets after rotation
    rotated_x_offset = x_offset * math.cos(math.radians(rotation2)) - y_offset * math.sin(math.radians(rotation2))
    rotated_y_offset = x_offset * math.sin(math.radians(rotation2)) + y_offset * math.cos(math.radians(rotation2))

    # Paste the second image onto the transparent background with rotated x and y offsets
    bg.paste(img2, (int(rotated_x_offset), int(rotated_y_offset)), mask=img2)

    # Save the final image
    bg.show()

# Example usage
images_paths = [f"..\\data\\group_45\\RPf_00368_intact_mesh.png",f"..\\data\\group_45\\RPf_00370_intact_mesh.png"]
paste_images(images_paths[0], images_paths[1], 50, 100, rotation1=30, rotation2=-15)


In [39]:
#from PIL import Image
#import math

def paste_images(base_image, overlay_image, offsets, rotations):
    """
    Paste two overlay images onto a base image with given offsets and rotations.

    Args:
    - base_image (PIL.Image.Image): The base image with transparent background.
    - overlay_image (list of str): List of file paths of the overlay images with transparent backgrounds.
    - offsets (list of tuples): List of (x, y) offsets for each overlay image.
    - rotations (list of float): List of rotation angles in degrees for each overlay image.

    Returns:
    - Image: The resulting image with overlay images pasted onto the base image.
    """
    for i in range(len(overlay_image)):
        # Open the overlay image
        img = Image.open(overlay_image[i])

        # Rotate the overlay image
        img = img.rotate(rotations[i], expand=True)

        # Calculate the x and y offsets after rotation
        rotated_x_offset = offsets[i][0] * math.cos(math.radians(rotations[i])) - offsets[i][1] * math.sin(math.radians(rotations[i]))
        rotated_y_offset = offsets[i][0] * math.sin(math.radians(rotations[i])) + offsets[i][1] * math.cos(math.radians(rotations[i]))

        # Paste the overlay image onto the base image with rotated x and y offsets
        base_image.paste(img, (int(rotated_x_offset), int(rotated_y_offset)), mask=img)

    return base_image


# Example usage
base_image = Image.new('RGBA', (4885, 4300), (0, 0, 0, 0))
overlay_images = [f"..\\data\\group_45\\RPf_00368_intact_mesh.png",f"..\\data\\group_45\\RPf_00370_intact_mesh.png"]

offsets = []
rotations = []
sins = []
coss = []
for trans in tranformations:
    #piece_id = int(trans[0])
    t_x = base_image.witrans[1]#*200
    t_y = trans[2]#*200
    offsets.append((t_x,t_y))
    #rotations.append(np.degrees(np.arccos(trans[3])))
    rotations.append(math.acos(-trans[3]))
    #coss.append(trans[3])
    #sins.append(trans[4])

#offsets = [(tranformations[0][], 100), (-30, 80)]
# for rot in tra
# rotations = [30, -15]

result = paste_images(base_image, overlay_images, offsets, rotations)
result.show('result.png')
